### Text cleaning and topic modeling

This notebook is an example of topic modeling adapted from [this writeup](https://medium.com/@sayahfares19/text-analysis-topic-modelling-with-spacy-gensim-4cd92ef06e06).

It performs the following tasks:

- the first part of the notebook loads texts from a spreadsheet and turns them into one large corpuse
- then we walk through various ways in which we can analyze and clean our corpus using spaCy (this includes taking out `stopwords` — words most often used in the English language and lemmatizing our corpus)
- to better understand how a model works this notebook also explores some funcationalities of spaCy
- the last parts of this notebook then make a simple topics model from the cleaned language data

The libraries we will use are:
- `pandas`: for reading in and exporting spreadsheets
- `spacy`: a natural language processing library that contains various models trained on various languages
- `gensim`: a library for topic modelling, document indexing and similarity retrieval with large corpora. In this case we will use it for topic modeling, the process of clustering words that seem to be used a lot in relation to one another. The algorithms built into genim that this notebook uses are called [Latent Dirichlet Allocation (LDA)](https://towardsdatascience.com/latent-dirichlet-allocation-lda-9d1cd064ffa2) and [Latent Semantic Analysis (LSA)](https://blog.marketmuse.com/glossary/latent-semantic-analysis-definition/).
- `pyLDAvis`: a library that is capable of visualizing your topics clusters.

Topic modeling is a form of unsupervised machine learning and can be really helpful in discovering topics in a large amount of text, especially if you're uncertain which topics might be buried in thousands or millions of documents. 

In [1]:
import os 
import pandas as pd

# for comprehension of language
import spacy 
from spacy import displacy

# for topics modeling
import gensim 
from gensim.corpora import Dictionary
from gensim.models import LdaModel, CoherenceModel, LsiModel, HdpModel


### Load spaCy's English language trained pipeline

`A training pipeline typically reads training data from a feature store, performs model-dependent transformations, trains the model, and evaluates the model before the model is saved to a model registry.`

You will need to download one of spaCy's models and can do so by typing this into a cell here:
```
!python3 -m spacy download en_core_web_sm

```

In [2]:
# !python3 -m spacy download en_core_web_sm

In [3]:
#load the English language model 
nlp = spacy.load('en_core_web_sm')

#### Stop words

A lot of languages also contain 'stop words', words that are used very frequently and may not be useful when we're evaluating how often certain words may be used. spaCy has niftyfunctions that allow us to designate stop words for our analysis. 

For this purpose, we got stopwords [here](https://gist.github.com/sebleier/554280).

First we need to open the text file adn then turn it into a list of words:

In [4]:

with open("../data/stopwords.txt", "r") as file:
    stop_words = file.read().split("\n")

print(
    len(stop_words), 
    stop_words)

430 ['a', 'about', 'above', 'across', 'after', 'again', 'against', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'among', 'an', 'and', 'another', 'any', 'anybody', 'anyone', 'anything', 'anywhere', 'are', 'area', 'areas', 'around', 'as', 'ask', 'asked', 'asking', 'asks', 'at', 'away', 'b', 'back', 'backed', 'backing', 'backs', 'be', 'became', 'because', 'become', 'becomes', 'been', 'before', 'began', 'behind', 'being', 'beings', 'best', 'better', 'between', 'big', 'both', 'but', 'by', 'c', 'came', 'can', 'cannot', 'case', 'cases', 'certain', 'certainly', 'clear', 'clearly', 'come', 'could', 'd', 'did', 'differ', 'different', 'differently', 'do', 'does', 'done', 'down', 'down', 'downed', 'downing', 'downs', 'during', 'e', 'each', 'early', 'either', 'end', 'ended', 'ending', 'ends', 'enough', 'even', 'evenly', 'ever', 'every', 'everybody', 'everyone', 'everything', 'everywhere', 'f', 'face', 'faces', 'fact', 'facts', 'far', 'felt', 'few', 'find', 'finds', 'f

Next we use spaCy's model and define stopwords:

In [5]:

# starts a loop that iterates through each word in the stop_words list.
for stopword in stop_words:
    # This line retrieves the lexeme (the base or dictionary form of a word) from spaCy's vocabulary. 
    lexeme = nlp.vocab[stopword]
    # then we set `lexeme.is_stop = True`for each word, making every word a stop word in spaCy's vocabulary.
    lexeme.is_stop = True


## Loading your text and making it a corpus

#### First we need to load the text

In [6]:
#load a spreadsheet with the text you want to analyze
tiktok_influencer =  pd.read_csv("../data/transcripts_demure.csv")

print(len(tiktok_influencer))
tiktok_influencer.head()

7


,file_name,transcript
0,../data/@joolieannie_7404929915893681451.mp4,"Ladies, let's be mindful when we use our phon..."
1,../data/@joolieannie_1724362477829.mp4,give me one that's like the size of like a fi...
2,../data/@joolieannie_1723610748575.mp4,You see how I do my makeup for work? Very dem...
3,../data/@joolieannie_1724324953244.mp4,Divas I'm in Los Angeles and Zillow needs my ...
4,../data/@joolieannie_1724362572281.mp4,"Hi, Tvaz. Okay, so I've been going to the sam..."


The next lines take all content from the `transcript` column, turn it into a list and then join it all with a space between each text. This creates one large corpus:

In [7]:
transcript_list = tiktok_influencer["transcript"].tolist()
 
text = ' '.join(str(x) for x in transcript_list)

In [8]:
len(text)

4104

In [9]:
doc = nlp(text)


In [10]:
len(doc)

1015

## Data cleaning
The next few lines 'normalize' the text and turns words into lemmas, get rid of stopwords and punctuation markers, and add lemmatized words.

In [11]:
# here's a demo of us cycling through the 
for word in doc:
    print(f"the lemma for the word {word} is {word.lemma_}")

the lemma for the word   is  
the lemma for the word Ladies is Ladies
the lemma for the word , is ,
the lemma for the word let is let
the lemma for the word 's is us
the lemma for the word be is be
the lemma for the word mindful is mindful
the lemma for the word when is when
the lemma for the word we is we
the lemma for the word use is use
the lemma for the word our is our
the lemma for the word phones is phone
the lemma for the word . is .
the lemma for the word You is you
the lemma for the word know is know
the lemma for the word me is I
the lemma for the word , is ,
the lemma for the word I is I
the lemma for the word keep is keep
the lemma for the word it is it
the lemma for the word very is very
the lemma for the word cutesy is cutesy
the lemma for the word , is ,
the lemma for the word very is very
the lemma for the word demure is demure
the lemma for the word . is .
the lemma for the word I is I
the lemma for the word reply is reply
the lemma for the word to is to
the lemma for 

In [12]:
# We add some words to the stop word list

#let's create some empty arrays. 
# texts will hold all our words that we will use for our topic model
texts = []
# is a temporary array that we will use to store the lemma-version of a word
article = []

for word in doc:
    if word.text != '\n' and not word.is_stop and not word.is_punct and not word.like_num:
        article.append(word.lemma_)
        texts.append(article)
        article = []
        
print(texts[1], len(texts))

['Ladies'] 279


In [13]:
texts

[[' '],
 ['Ladies'],
 ['mindful'],
 ['phone'],
 ['cutesy'],
 ['demure'],
 ['reply'],
 ['text'],
 ['email'],
 ['picky'],
 ['picky'],
 ['flicky'],
 ['flicky'],
 ['flicky'],
 ['Verizon'],
 ['Verizon'],
 ['trade'],
 ['musty'],
 ['diva'],
 ['demure'],
 ['diva'],
 ['Verizon'],
 ['trade'],
 ['crusty'],
 ['phone'],
 ['crunchy'],
 ['phone'],
 ['crack'],
 ['screen'],
 ['type'],
 ['phone'],
 ['get'],
 ['cut'],
 ['finger'],
 ['charge'],
 ['phone'],
 ['crazy'],
 ['thank'],
 ['bag'],
 ['bag'],
 ['cute'],
 ['respectful'],
 ['staff'],
 ['crazy'],
 ['walk'],
 ['nice'],
 ['phone'],
 ['partner'],
 ['Verizon'],
 ['elegant'],
 ['cutesy'],
 ['classy'],
 ['red'],
 ['hot'],
 ['pink'],
 ['crazy'],
 ['demure'],
 [' '],
 ['size'],
 ['calibrate'],
 ['bucket'],
 [' '],
 ['makeup'],
 ['demure'],
 ['mindful'],
 ['green'],
 ['cut'],
 ['crease'],
 ['look'],
 ['clown'],
 ['mindful'],
 ['look'],
 ['presentable'],
 ['interview'],
 ['job'],
 ['lot'],
 ['girl'],
 ['interview'],
 ['look'],
 ['Marge'],
 ['Simpson'],
 ['job']

In the next lines we turn these cleaned texts into a bag-of-words format.


In [14]:
# Dictionary() to map each unique word to a unique integer ID
dictionary = Dictionary(texts)
# this line creates a corpus and converts a single document (a list of words) into a bag-of-words format
corpus = [dictionary.doc2bow(text) for text in texts]

print(corpus[1])

[(1, 1)]


## Different Kinds of Topic Modeling 
Topic Modeling refers to the probabilistic modeling of text documents as topics. Gensim is one of the most popular libraries to perform such modeling.

#### LSI — Latent Semantic Indexing
One of the methods available in gensim is called LSI, which stands for Latent Semantic Indexing. LSI aims to find hidden (latent) relationships between words and concepts in a collection of documents. The assumption here is words that are used in similar contexts tend to have similar meanings.

In [15]:
lsi_model = LsiModel(corpus=corpus, num_topics=10, id2word=dictionary)
lsi_model.show_topics(num_topics=15)

[(0,
  '1.000*"demure" + -0.000*"post" + -0.000*"treat" + -0.000*"help" + -0.000*"check" + 0.000*"Netflix" + -0.000*"get" + 0.000*"Ladies" + 0.000*"shirt" + -0.000*"availability"'),
 (1,
  '1.000*"mindful" + -0.001*"type" + 0.001*"bang" + -0.001*"Um" + -0.001*"help" + 0.001*"staff" + -0.001*"chocho" + -0.001*"little" + -0.001*"cheaty" + 0.000*"Patty"'),
 (2,
  '0.837*"watch" + -0.489*" " + 0.215*"house" + -0.117*"look" + 0.001*"bush" + -0.001*"treat" + 0.001*"water" + -0.001*"okay" + 0.001*"Divas" + 0.001*"bang"'),
 (3,
  '0.744*"look" + 0.447*" " + 0.427*"house" + 0.255*"watch" + -0.002*"clown" + -0.002*"skillful" + 0.001*"find" + -0.001*"Wavuzva" + -0.001*"hot" + 0.001*"classy"'),
 (4,
  '-0.610*" " + 0.597*"house" + -0.484*"watch" + 0.190*"look" + -0.002*"pink" + -0.002*"Los" + 0.002*"Divas" + 0.001*"charge" + -0.001*"bush" + -0.001*"crunchy"'),
 (5,
  '-0.644*"house" + 0.630*"look" + -0.434*" " + 0.003*"post" + -0.003*"Selma" + 0.002*"chose" + 0.002*"party" + 0.002*"treat" + -0.002

#### HDP — Hierarchical Dirichlet Process
HDP, the Hierarchical Dirichlet Process is an unsupervised Topic Model which figures out the number of topics on its own. HPD assumes that documents are mixtures of topics, and topics are mixtures of words, but it doesn't limit the number of topics.

In [16]:
hdp_model = HdpModel(corpus=corpus, id2word=dictionary)
hdp_model.show_topics()[:5]

[(0,
  '0.039*party + 0.028*prize + 0.022*partner + 0.021*classy + 0.020*tech + 0.019*invest + 0.019*elegant + 0.019*bed + 0.019*Los + 0.018*availability + 0.018*crunchy + 0.017*water + 0.016*type + 0.015*mean + 0.015*Hi + 0.015*presentable + 0.014*obviously + 0.014*balcony + 0.014*glibadi + 0.014*mix'),
 (1,
  '0.035*Hollywood + 0.027*Patty + 0.026*hot + 0.026*blinker + 0.024*crack + 0.023*forever + 0.021*Wavuzva + 0.020*murid + 0.019*mindful + 0.019*girl + 0.019*Diva + 0.017*glibadi + 0.017*drive + 0.017*eat + 0.016*watch + 0.016*chocho + 0.016*course + 0.015*cheesy + 0.015*tomorrow + 0.015*bucket'),
 (2,
  '0.031*respectful + 0.025*crunchy + 0.023*bring + 0.019*live + 0.017*crack + 0.017*pink + 0.016*availability + 0.016*tile + 0.016*hygienic + 0.016*bag + 0.016*watch + 0.015*mindful + 0.015*episode + 0.015*charge + 0.015*time + 0.014*Willa + 0.014*noise + 0.014*look + 0.014*balcony + 0.014*chocho'),
 (3,
  '0.044*screen + 0.031*favorite + 0.031*mix + 0.029*post + 0.027*walk + 0.021

#### LDA — Latent Dirichlet Allocation
LDA or Latent Dirichlet Allocation is arguably the most famous Topic Modeling algorithm out there. Out here we create a simple Topic Model with 5 topics. The LDA algorithm assumes that each document is a mixture of topics, and each topic is a mixture of words.

In [17]:
lda_model = LdaModel(corpus=corpus, num_topics=5, id2word=dictionary)
lda_model.show_topics()

[(0,
  '0.055*"watch" + 0.044*"ugly" + 0.044*"elegant" + 0.034*"cozy" + 0.034*"beddy" + 0.024*"phone" + 0.023*"simple" + 0.023*"bag" + 0.023*"ice" + 0.023*"walk"'),
 (1,
  '0.072*" " + 0.072*"look" + 0.052*"mindful" + 0.052*"cute" + 0.052*"real" + 0.042*"phone" + 0.042*"girl" + 0.022*"Diva" + 0.022*"cut" + 0.012*"go"'),
 (2,
  '0.062*"diva" + 0.050*"crazy" + 0.026*"watch" + 0.026*"house" + 0.026*"cutesy" + 0.026*"glibadi" + 0.026*"tomorrow" + 0.026*"tag" + 0.026*"morning" + 0.026*"interview"'),
 (3,
  '0.064*"demure" + 0.051*"nail" + 0.039*"seey" + 0.027*"mindful" + 0.027*"cutesy" + 0.027*"let" + 0.027*"wild" + 0.027*"sweet" + 0.027*"to" + 0.027*"Willa"'),
 (4,
  '0.088*"demure" + 0.063*"house" + 0.051*"Verizon" + 0.039*"mindful" + 0.039*"tech" + 0.039*"beautiful" + 0.039*"flicky" + 0.027*"nail" + 0.027*"nice" + 0.027*"job"')]

## Visualizing Topics with pyLDAvis
pyLDAvis is designed to help users interpret the topics in a topic model that has been fit to a corpus of text data. The package extracts information from a fitted LDA topic model to inform an interactive web-based visualization.

The visualization is intended to be used within an IPython notebook but can also be saved to a stand-alone HTML file for easy sharing.

**Note: If you have issues running the `pyLDAvis.gensim_modeles.prepare()` function, you may need to walk yourself through [these fixes](https://docs.google.com/document/d/1XOz5fJdHR754SHkMIrqCxkgD2yGHhBlueK4EcbaNwII/edit?usp=sharing).**

In [18]:
#for visualizations
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()


In [19]:
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)


In [20]:
pyLDAvis.save_html(vis, "../output/topics_modeling_demure.html")

In [21]:
# vis

## Word count
For good measure, we can also use this space to make a word count

In [22]:
words_influencer = pd.DataFrame(texts)
print(len(words_influencer))
words_influencer.columns = ["word"]
words_influencer.head()


279


,word
0,
1,Ladies
2,mindful
3,phone
4,cutesy


In [23]:
word_tally = words_influencer["word"].value_counts()
word_tally.head()

word
demure     12
mindful    10
            7
look        7
house       7
Name: count, dtype: int64

In [24]:
word_tally.to_csv("../output/word_tally_demure.csv")